In [1]:
# Esto agrega al python path el directorio ..
import sys
sys.path.append("..")

import numpy as np
import config


from dataframe_builder import DataFrameBuilder
# Ignorar Warning de sklearn (no es importante)
import warnings
warnings.filterwarnings('ignore',
                        message='Changing the shape of non-C contiguous array')


print "Creando dataframe..."


builder = DataFrameBuilder()
df = builder.build()

print "Listo"

Creando dataframe...
Listo



Conjunto a probar

1)C in range [0:20] Dice el manual:Por default es uno. If you have a lot of noisy observations you should decrease it. It corresponds to regularize more the estimation.

2)Kernels {'rbf ' ,' linear' ,' poly' ,'sigmoid'}  

3) para poly podemos hacer un range de degrees, esa opcion va a ser ignorada por los otros kernels asi que podemos hacer degree in range[2:5]. Podemos elegis=r ademas si agregamos un coef0, o sea el termino independiente, no veo por que seria importnte por ahora eso.

4)(REVISAR)Por default hay un parametro epsilon=0.1. Epsilon indica un rango de distancia que no sera considerado como penalidad, si el vector esta a espilon--ver

decision_function(X)[source]
Distance of the samples X to the separating hyperplane.
Parameters:	
X : array-like, shape (n_samples, n_features)
returns:	
X : array-like, shape (n_samples, n_classes * (n_classes-1) / 2)
Returns the decision function of the sample for each class in the model. If decision_function_shape=’ovr’, the shape is (n_samples, n_classes)

-Cuando fiteamos este modelo podemos elegir un sample weight, pero no creo que lo necesitemos ya que estan balanceados.



In [2]:
from sklearn.svm import SVC
from sklearn.metrics.pairwise import chi2_kernel
from sklearn.metrics.pairwise import laplacian_kernel


options={
    'kernel' :['rbf ',' linear', 'poly', 'sigmoid'], # poly tarda mucho, por ahi seria mejor hacerlo aparte
    'C' : range(1,10),
    'degree' : range(0,4),
    'gamma' : np.arange(0,1,0.1)
}

options_2={
    'kernel' :[chi2_kernel, laplacian_kernel],
    'C' : range(1,10),
    'gamma' : np.arange(0,1,0.1)
}

Pendiente: buscar un kernel adecuado, mas alla de las opciones que da sklearn. Recordar para el informe poner la cantidad de vectores de soporte, esos son los vectores sobre los cuales esta basado el hiperplano. Se puede obtener con los atributos.

En el manual de sklearn dice: Proper choice of C and gamma is critical to the SVM’s performance. One is advised to use sklearn.grid_search.GridSearchCV with C and gamma spaced exponentially far apart to choose good values.


Attributes:	
n_support_ : array-like, dtype=int32, shape = [n_class]
Number of support vectors for each class.


In [ ]:
from sklearn.grid_search import RandomizedSearchCV, GridSearchCV
# Preparo data para clasificar
X = df.design_matrix
y = df.outcomes

print X[:10], y[:10]

     len  spaces  has_html  has_original_message  has_multipart  has_dear  \
0  21272    1195         1                     0              1         0   
1   1385      48         1                     0              0         0   
2   2282     158         1                     0              0         0   
3   2734     268         1                     0              1         0   
4   1024     112         1                     0              0         0   
5   2695     175         1                     0              0         0   
6   1732     115         1                     0              0         0   
7   3013     485         0                     0              1         1   
8   6123     141         1                     0              1         0   
9   4794     447         1                     0              0         0   

   has_Friend  has_hello  has_$  has_earn      ...        has_bill  \
0           0          0     13         0      ...               0   
1           

In [ ]:
from sklearn.grid_search import RandomizedSearchCV, GridSearchCV
# Preparo data para clasificar
X = df.design_matrix
y = df.outcomes

scoring_methods = ['precision', 'accuracy', 'f1', 'recall', 'roc_auc']

for scoring in scoring_methods:
    clf = SVC()

    print("=" * 80 + "\n")
    print("Scoring {}".format(scoring))

    #search = GridSearchCV(clf, scoring=scoring, param_grid=options, n_jobs=4)
    search = RandomizedSearchCV(clf, scoring="f1", param_distributions=options, cv=10, n_jobs=4, n_iter=2)
        
    search.fit(X, y)
    
    print "Mejor combinación: {}".format(search.best_params_)
    print "Mejor valor: {}\n\n".format(search.best_score_)

    """
    Esto puede no tener mucho sentido. Estamos corriendo nuevamente cross_val_score... 
    """
    for other_scorer in scoring_methods:
        print "Valor para {} = {}".format(
            other_scorer,
            cross_val_score(search.best_estimator_, X, y, scoring=other_scorer, cv=10).mean()
        )
        
    


Scoring precision
